<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/1.0-la-tj-ak-ensemble_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [9]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.2.0
/bin/bash: 1.14: No such file or directory


In [1]:
# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'


# Install required packages 
!pip install -r  '/content/cs231n-emotiw/requirements.txt'


fatal: destination path 'cs231n-emotiw' already exists and is not an empty directory.
     |████████████████████████████████| 92.6MB 1.2MB/s 
     |████████████████████████████████| 368kB 48.3MB/s 
     |████████████████████████████████| 3.2MB 55.0MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2


In [4]:
import tensorflow
tensorflow.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.2'

#### Navigate to the repo we downloaded
We will run all our commands from this repo

In [5]:
!pwd
import os
os.chdir('/content/cs231n-emotiw')
!pwd

/content
/content/cs231n-emotiw


#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

In [6]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

--2020-05-29 14:55:47--  https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.128.128, 2a00:1450:4013:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 598708936 (571M) [application/x-tar]
Saving to: ‘openpose.tar.gz.1’

openpose.tar.gz.1   100%[===================>] 570.97M   133MB/s    in 4.3s    

2020-05-29 14:55:51 (133 MB/s) - ‘openpose.tar.gz.1’ saved [598708936/598708936]



#### Set up Google Drive and GCS
(if needed)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [6]:
!wget https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/test-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt

--2020-05-29 14:39:56--  https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50131084 (48M) [application/zip]
Saving to: ‘train-tiny.zip’

train-tiny.zip      100%[===================>]  47.81M  29.3MB/s    in 1.6s    

2020-05-29 14:39:58 (29.3 MB/s) - ‘train-tiny.zip’ saved [50131084/50131084]

--2020-05-29 14:40:00--  https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48020782 (46M) [application/zip]
Saving to: ‘val-tiny.zip’

val-tiny.zip        100%[==========

#### Pre-Processing

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [7]:
from src.preprocessors.scene_preprocessor import VideoPreprocessor
from src.preprocessors.face_preprocessor import FacePreprocessor
from src.preprocessors.pose_preprocessor import PosePreprocessor
from src.preprocessors.audio_preprocessor import AudioPreprocessor

video_preprocessor = VideoPreprocessor(
    video_folder= "train-tiny.zip", 
    label_file= "Train_labels.txt", 
    output_folder="train-tiny-local", 
    output_file= "train-tiny-local.zip"
)

face_preprocessor = FacePreprocessor(
    video_folder="train-tiny.zip",
    output_folder="train-tiny-faces", 
    output_file="train-tiny-faces.zip"
)

pose_preprocessor = PosePreprocessor(
    video_frame_folder="val-tiny.zip",
    output_folder="val-tiny-pose", 
    output_file="val-tiny-pose.zip"
)

audio_preprocessor = AudioPreprocessor(
    output_folder="train-tiny-audio", 
    output_file= "train-tiny-audio.zip" ,
    video_folder= "train-tiny.zip",
    label_path = "Train_labels.txt"
)

preprocessors_list = [video_preprocessor, face_preprocessor, pose_preprocessor, audio_preprocessor] 

for preprocessor in preprocessors_list:
    preprocessor.preprocess()


Using TensorFlow backend.


Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , label_file = Train_labels.txt , output_folder = train-tiny-local, output_file = train-tiny-local.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , output_folder = train-tiny-faces, output_file = train-tiny-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Pose Preprocessor created with is_zip = True, is_test = False, video_frame_folder = val-tiny.zip , output_folder = val-tiny-pose, output_file = val-tiny-pose.zip
Video Preprocessor created with is_zip = True, video_folder = train-tiny.zip , output_folder = train-tiny-audio, output_file = train-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir train-tiny-local_tmp...
Finished unzipping files
Found 50 videos
Processing video 1/50 with name 101_12.mp4 and class 3 

Processing video 4/

#### Predictions
After performing pre-processing


In [0]:
def predict(mp4_dir , train_target_path , model_list= [model,] , model_paths=["",] , num_models = 1 , mode="soft" , complexFusion=False):

  """

  Notes: 

  All file preprocessing will occur in this function.


  Inputs
  
  * mp4_train_dir - The directory of .mp4 file paths that the model will make predictions from.
  * train_target_path - The target path. Should be in .txt format.
  * A list of pretrained models that will be used in the ensemble during prediction


  Outputs 

  * An array of size (M) where M is the number of samples

  """

  assert len(model_list) == num_models
  assert len(model_paths) == num_models

  all_model_predictions = []
  X_list = []

  counter = 0

  audio = AudioPreprocessor()
  face = 

  # for model in models:
  #    # Do preprocess here on a model basis.
  #   X =  model.preprocess("video path/*.mp4") # X Can either be a np array or None. Can either read from disk, work with raw files, or return None
  #                                             # if using a data generator.
  #   X_list.append(X)                             

  for model in models:

    model.load_model(model_paths[counter])


    X = X_list[counter]
    
   

    all_model_predictions.append(model.predict(dir="video path/*.mp4" , X=X)) # Predict returns an (M , 3) array
    
    counter += 1

  all_model_predictions = np.asarray(all_model_predictions , dtype='float32') # (num_models , M , 3)


  assert mode in ["soft" , "hard"]

  if mode == "soft":

    
    # Take the average of each 

    predictions = np.mean(all_model_predictions , axis=0)

    predictions = np.argmax(predictions , axis = 1)

    return predictions  # (M,) in the domain [0,1,2]

  # TODO: Majority vote

  positive_hard = np.argmax(positive_arr , axis=1) # ()



  return predictions 

In [0]:
def evaluate(predictions, targets):

  # TO DO: Add evaluate code here

  """
  Inputs
  
  * predictions np array of shape M where M is the number of samples
  * target array of shape M where M is the number of samples


  Outputs 

  * Model accuracy scalar 

  """

  assert len(targets) == len(predictions)

  incorrect = np.count_nonzero(predictions - targets)

  acc = (targets.shape[0] - incorrect) / targets.shape[0]

  return acc


